# Evaulate Distilbert TopicChooser Model
### 29 Nov 2020

## I. Evaluation Parameters

In [1]:
# Set Evaulation Parameters
MODEL_NAME = "allenai/longformer-base-4096"
MODEL_FILE ="saved_model_epoch10_20201130_0855.tar"
VAL_DATASET = "val_data_29Nov_longformer_pf.pickle"
EVAL_BATCH_SIZE = 8
DESCRIPTION = "PolitiFact only model"

## II. Setup and Load Model

In [2]:
import pickle
import sys

import json
import pandas as pd
import pickle
import sklearn.metrics
import tqdm
import torch
import transformers as hft

sys.path.insert(0, "/home/jupyter")
import util.log
import util.data

In [3]:
# Load evaluation data
with open(VAL_DATASET, "rb") as vfile:
    val_dataset = pickle.load(vfile)

In [4]:
# Load saved model
checkpoint = torch.load(MODEL_FILE)
model = (hft.LongformerForSequenceClassification
         .from_pretrained(MODEL_NAME))
model.load_state_dict(checkpoint["model"])

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', '

<All keys matched successfully>

In [5]:
# Move model to GPU if available
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
model.to(device)
model.eval();
device

device(type='cuda')

## III. Evaluation Loop and Results Dataframe

In [6]:
# Evaulate model
eval_loader = torch.utils.data.DataLoader(val_dataset,
                                batch_size=EVAL_BATCH_SIZE,
                                shuffle=False)
labels = []
preds = []
logits = []
probs = []
sources = []
article_tokens = []
file_names = []

softmax = torch.nn.Softmax(dim=1)

with torch.no_grad():
    for art_num, article in tqdm.tqdm(enumerate(eval_loader)):
        input_ids = article["input_ids"].to(device)
        attent_mask = article["attention_mask"].to(device)
        output = model(input_ids,
                       attention_mask=attent_mask,
                       output_hidden_states=False,
                       output_attentions=False)
        prob = softmax(output[0].detach().cpu()).numpy()
        logit = output[0].detach().cpu().numpy()
        pred = [0 if lgt[0] > lgt[1] else 1 for lgt in logit]
        label = article["labels"].numpy()
        labels.extend(label)
        preds.extend(pred)
        logits.extend(logit)
        probs.extend(prob)
        article_tokens.extend(article["article_tokens"].numpy())
        sources.extend(article["sources"])
        file_names.extend(article["file_names"])

17it [00:17,  1.06s/it]


In [7]:
# Label of 0 is real, 1 is fake
eval_results = pd.DataFrame({"Article": list(range(len(labels))),
                             "Source": sources,
                             "Token_Length": article_tokens,
                             "Predictions": preds,
                             "Labels": labels,
                             "Logit_real(0)": [x[0] for x in logits],
                             "Logit_fake(1)": [x[1] for x in logits],
                             "Prob_real(0)": [x[0] for x in probs],
                             "Prob_real(1)": [x[1] for x in probs],
                             "File_Name": file_names})

eval_results

,Article,Source,Token_Length,Predictions,Labels,Logit_real(0),Logit_fake(1),Prob_real(0),Prob_real(1),File_Name
0,0,politifact,429,0,0,4.000608,-3.126282,0.999197,0.000803,politifact13132
1,1,politifact,4579,0,0,4.025331,-3.151489,0.999236,0.000764,politifact348
2,2,politifact,574,1,1,-3.030061,2.121160,0.005759,0.994241,politifact13806
3,3,politifact,527,1,1,-3.756790,2.692146,0.001580,0.998420,politifact14954
4,4,politifact,481,0,0,4.177752,-3.228481,0.999393,0.000607,politifact1442
...,...,...,...,...,...,...,...,...,...,...
130,130,politifact,559,1,1,-3.635628,2.597688,0.001959,0.998041,politifact13751
131,131,politifact,1107,0,0,3.550458,-2.666180,0.998008,0.001992,politifact4588
132,132,politifact,243,1,1,-3.631677,2.608004,0.001947,0.998053,politifact14161
133,133,politifact,260,1,1,-3.543155,2.562153,0.002226,0.997774,politifact13775


In [8]:
# Save evaluation data to csv file
eval_results.to_csv("eval_data_" + MODEL_FILE[12:-4] + ".csv")

## IV. Overall Metrics

In [9]:
# Overall metrics
precision, recall, f1, _ = sklearn.metrics.precision_recall_fscore_support(
    eval_results.Labels, eval_results.Predictions, average="binary")
accuracy = sklearn.metrics.accuracy_score(eval_results.Labels, eval_results.Predictions)

metrics = {"description": DESCRIPTION,
           "eval_notes": "Evaluating PolitiFact data",
           "model": MODEL_FILE,
           "epochs": checkpoint["epoch"],
           "batch_size": EVAL_BATCH_SIZE,
           "Date_evaluated": "28Nov2020",
           "train_data": "train_data_29Nov_longformer_pf.pickle",
           "val_data": "eval_data_" + MODEL_FILE[12:-4] + ".csv",
           "eval_data": VAL_DATASET,
           "accuracy": accuracy,
           "precision": precision,
           "recall": recall,
           "f1": f1}
metrics

{'description': 'PolitiFact only model',
 'eval_notes': 'Evaluating PolitiFact data',
 'model': 'saved_model_epoch10_20201130_0855.tar',
 'epochs': 10,
 'batch_size': 8,
 'Date_evaluated': '28Nov2020',
 'train_data': 'train_data_29Nov_longformer_pf.pickle',
 'val_data': 'eval_data_epoch10_20201130_0855.csv',
 'eval_data': 'val_data_29Nov_longformer_pf.pickle',
 'accuracy': 0.8814814814814815,
 'precision': 0.9047619047619048,
 'recall': 0.8507462686567164,
 'f1': 0.8769230769230769}

In [10]:
# Save metrics to JSON text file.
with open("eval_metrics_" + MODEL_FILE[12:-4] + ".json", "wt") as jfile:
    json.dump(metrics, jfile)

In [11]:
cm = pd.DataFrame(sklearn.metrics.confusion_matrix(eval_results.Labels, eval_results.Predictions))
label_titles = {0: "real", 1: "fake"}
cm.rename(index=label_titles, columns=label_titles, inplace=True)
cm

,real,fake
real,62,6
fake,10,57
